<a href="https://colab.research.google.com/github/IshaqKHATTAK/Natural-Language-Processing/blob/main/Next_word_prediction_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#load data
#basic NLP preprocessing
#convert to integer using keras tokenizer
#make x and y by making firt word in the corpus as x and teh second as y
#make one hot encoding of y over all the vocabulay size
#use ebddings and build RNN or LSTM

In [2]:
from google.colab import files
video = files.upload()

Saving kaggle.json to kaggle.json


In [3]:
!mkdir -p ~/.kaggle

In [4]:
!cp kaggle.json ~/.kaggle

In [5]:

!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download imbikramsaha/poems

  0% 0.00/57.6k [00:00<?, ?B/s]
100% 57.6k/57.6k [00:00<00:00, 59.9MB/s]


In [7]:
!unzip '/content/poems.zip'

Archive:  /content/poems.zip
  inflating: poems-100.csv           


In [8]:
import pandas as pd
import numpy as np
df = pd.read_csv('/content/poems-100.csv')

In [9]:
df

,text
0,"O my Luve's like a red, red rose\nThat’s newly..."
1,"The rose is red,\nThe violet's blue,\nSugar is..."
2,How do I love thee? Let me count the ways.\nI ...
3,"Had I the heavens' embroidered cloths,\nEnwrou..."
4,"I.\n Enough! we're tired, my heart and I.\n..."
...,...
95,The city had withdrawn into itself\nAnd left a...
96,O gift of God! O perfect day:\n Whereon...
97,"The world is too much with us; late and soon,\..."
98,To him who in the love of Nature holds\nCo...


#preprocessing

In [10]:
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.6/356.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.6.0-py2.py3-none-any.whl size=351311 sha256=dda3b8df29cb1e6ad1689618396fc84edb4e9050dd6a6c977db6eab7e8def67e
  Stored in directory: /root/.cache/pip/wheels/ea/0b/64/114bc939d0083621aa41521e21be246c888260b8aa21e6c1ad
Successfully built emoji


In [11]:
import re
import emoji
def clean(text):
  text = text.lower()
  text = re.sub(r'[^\s\w]','',text)
  text = re.sub(r'https?://\S+','',text) #remove links
  text = re.sub(r'[^A-Za-z0-9 ]+', '', text) #remove special character
  text = emoji.demojize(text, delimiters=('', ''))
  return text

In [12]:
df['text'] = df['text'].apply(clean)

In [13]:
# stop word remval -- steaming -- lemetization

import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
sw = stopwords.words('english')
ps = PorterStemmer()
wl = WordNetLemmatizer()

def tokenize_steam(text):
  text = word_tokenize(text)
  text = [word for word in text if word not in sw]
  text = [wl.lemmatize(word) for word in text]
  text = [ps.stem(word )for word in text]
  text = ' '.join(text)
  return text

tokenize_steam('This is IshaQ fr fdj')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


'thi ishaq fr fdj'

In [14]:
df['text'] = df['text'].apply(tokenize_steam)

#formate the dataset and encode the text

In [15]:
# convert to numbers then padding
import keras
from keras.preprocessing.text import Tokenizer
from keras.layers import SimpleRNN
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [16]:
# integer encode text
tokenizer = Tokenizer()

In [17]:
tokenizer.fit_on_texts(df['text'])

In [18]:
len(tokenizer.word_index) #vocabulary has created wher the shape is word: idex

4985

In [19]:
sentences = tokenizer.texts_to_sequences(df['text'])

In [20]:
print(sentences) #this is df[0]


[[385, 7, 143, 143, 1635, 1636, 1637, 1638, 385, 7, 1639, 1640, 964, 1641, 268, 205, 36, 1642, 1643, 230, 385, 1644, 385, 23, 64, 1645, 62, 683, 1646, 62, 683, 269, 1647, 185, 1648, 1649, 965, 385, 23, 64, 1650, 509, 8, 3, 966, 386, 23, 37, 1651, 386, 23, 37, 1652, 9, 1653, 325, 106, 326], [684, 1654, 967, 1655, 1656], [1, 23, 50, 685, 1657, 1, 23, 510, 1658, 1659, 84, 144, 65, 1660, 206, 1661, 1662, 1, 23, 511, 19, 1663, 512, 231, 51, 1664, 1, 23, 686, 28, 513, 1665, 1, 23, 687, 107, 1666, 1, 23, 688, 73, 1667, 16, 207, 968, 1668, 1, 23, 1, 117, 1669, 145, 969, 1, 1670, 514, 8, 32, 1671, 3, 1, 23, 232, 66], [55, 1672, 1673, 515, 689, 1674, 186, 387, 74, 1675, 14, 35, 1676, 20, 208, 327, 1677, 690, 1678, 208, 270, 1679, 970, 328, 270], [92, 93, 6, 146, 516, 1680, 329, 165, 209, 971, 29, 972, 1681, 691, 388, 1682, 973, 389, 55, 147, 8, 974, 21, 8, 93, 6, 1683, 5, 93, 6, 1684, 692, 271, 28, 148, 1685, 1686, 1687, 46, 187, 56, 693, 975, 206, 1, 272, 166, 149, 85, 93, 6, 1688, 93, 65, 6, 1

In [21]:
# determine the vocabulary size
vocab_size = len(tokenizer.word_index) + 1  # 0 is reserved for padding so that's why we added 1
print(vocab_size)

4986


#Next, we need to create sequences of words to fit the model with one word as input and one word as output.

In [22]:
len(sentences[0])

58

###making tri-grams for perdiction

In [23]:
data = []
for sent in sentences:
  for i in range(1, len(sent)):
    seq = sent[i-1:i+1]
    data.append(seq)

In [24]:
print(data[0:10])

[[385, 7], [7, 143], [143, 143], [143, 1635], [1635, 1636], [1636, 1637], [1637, 1638], [1638, 385], [385, 7], [7, 1639]]


In [25]:
data_array = np.array(data)

In [26]:
data_array.ndim

2

In [27]:
#conver it into x and y

X, y = data_array[:,0],data_array[:,1]

In [28]:
y[:20]

array([   7,  143,  143, 1635, 1636, 1637, 1638,  385,    7, 1639, 1640,
        964, 1641,  268,  205,   36, 1642, 1643,  230,  385])

In [29]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.layers import Embedding

In [30]:
#convert the ouputs into the one hot vector over all the unique words
y = to_categorical(y,num_classes=vocab_size)

In [31]:
y[:2]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

#model building and embeddings



*   first convert each word into embedding instead of just integer encoding
*   then build rnn model as shown



In [32]:
model = Sequential()

model.add(Embedding(vocab_size, 2, input_length = 1))

model.add(LSTM(100))

model.add(Dropout(0.2))

model.add(Dense(vocab_size, activation='softmax'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 2)              9972      
                                                                 
 lstm (LSTM)                 (None, 100)               41200     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4986)              503586    
                                                                 
Total params: 554,758
Trainable params: 554,758
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [34]:
model.fit(X, y, epochs=5)

Epoch 1/5
376/376 [==============================] - 20s 34ms/step - loss: 8.3637 - accuracy: 0.0062
Epoch 2/5
376/376 [==============================] - 11s 28ms/step - loss: 7.9525 - accuracy: 0.0061
Epoch 3/5
376/376 [==============================] - 7s 20ms/step - loss: 7.8439 - accuracy: 0.0060
Epoch 4/5
376/376 [==============================] - 7s 18ms/step - loss: 7.7664 - accuracy: 0.0061
Epoch 5/5
376/376 [==============================] - 6s 16ms/step - loss: 7.7065 - accuracy: 0.0067


In [46]:
def generate_text(model, tokenizer, enter_text, n_pred):
    in_text, result = enter_text, enter_text
    for _ in range(n_pred):
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        encoded = array(encoded)

        predicted = model.predict(encoded, verbose=0)
        yhat = np.argmax(predicted, axis=1)

        out_word = ''
        for word, index in tokenizer.word_index.items():
          if index == yhat:
            out_word = word
            break
        # append to input
        in_text, result = out_word, result + ' ' + out_word
    return result

In [49]:
print (generate_text(model, tokenizer, 'sad', 6))

sad exclud love one one one one


In [50]:
print (generate_text(model, tokenizer, 'touch', 6))

touch one one one one one one


In [51]:
print (generate_text(model, tokenizer, 'come', 6))

come one one one one one one
